In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=81704dccf37261f313c21705a07a21fe6c8efe6c6ac513d6062563d0b78d3f70
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


Task 1: Vehicle Maintenance Data Ingestion

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DateType
import os

# Create a Spark session
spark = SparkSession.builder.appName("VehicleMaintenanceIngestion").getOrCreate()

# Define the schema for the vehicle maintenance data
schema = StructType([
    StructField("VehicleID", StringType(), True),
    StructField("Date", DateType(), True),
    StructField("ServiceType", StringType(), True),
    StructField("ServiceCost", FloatType(), True),
    StructField("Mileage", FloatType(), True)
])

# Define paths
raw_data_path = "/content/sample_data/vehicle_maintenance.csv"
delta_table_path = "/content/sample_data/delta/vehicle_maintenance"

# Check if the raw data file exists
if os.path.exists(raw_data_path):
    try:
        # Read the CSV file into a DataFrame
        vehicle_df = spark.read.csv(raw_data_path, schema=schema, header=True).withColumn("file_name", input_file_name())

        # Write the DataFrame to a Delta table
        vehicle_df.write.format("delta").mode("overwrite").save(delta_table_path)
        print("Data loaded and saved as Delta table.")
    except Exception as e:
        print(f"Error: {e}")
else:
    print(f"File not found: {raw_data_path}")


Error: An error occurred while calling o67.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:240)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.refl

Task 2: Data Cleaning

In [ ]:
# Read the raw Delta table
raw_df = spark.read.format("delta").load(delta_table_path)

# Clean the DataFrame
cleaned_df = raw_df.filter((raw_df.ServiceCost > 0) & (raw_df.Mileage > 0))

# Remove duplicates based on VehicleID and Date
cleaned_df = cleaned_df.dropDuplicates(["VehicleID", "Date"])

# Save the cleaned data to a new Delta table
cleaned_delta_path = "/content/sample_data/delta/cleaned_vehicle_maintenance"
cleaned_df.write.format("delta").mode("overwrite").save(cleaned_delta_path)
print("Cleaned data saved to Delta table.")


Py4JJavaError: An error occurred while calling o71.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at jav

Task 3: Vehicle Maintenance Analysis

In [ ]:
# Read the cleaned data
cleaned_df = spark.read.format("delta").load(cleaned_delta_path)

# Calculate total maintenance cost for each vehicle
maintenance_costs = cleaned_df.groupBy("VehicleID").agg({"ServiceCost": "sum"}).withColumnRenamed("sum(ServiceCost)", "TotalMaintenanceCost")
maintenance_costs.show()

# Identify vehicles exceeding mileage threshold (30,000 miles)
threshold = 30000
vehicles_needing_service = cleaned_df.filter(cleaned_df.Mileage > threshold).select("VehicleID", "Mileage")
vehicles_needing_service.show()

# Save analysis results to Delta tables
maintenance_costs.write.format("delta").mode("overwrite").save("/content/sample_data/delta/maintenance_costs")
vehicles_needing_service.write.format("delta").mode("overwrite").save("/content/sample_data/delta/vehicles_needing_service")


Task 4: Data Governance with Delta Lake

In [ ]:
# Use VACUUM to clean up old data
spark.sql("VACUUM '/content/sample_data/delta/vehicle_maintenance' RETAIN 168 HOURS")

# Check the history of updates
history_df = spark.sql("DESCRIBE HISTORY delta.`/content/sample_data/delta/vehicle_maintenance`")
history_df.show()
